# Constructing the scattering potential of the sample

In [1]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import sys

sys.path.append("../")
from sample import (
    calc_extraordinary_refractive_index,
    dielectric_tensor_global,
    rotation_matrix_to_align_axes_single,
    rotation_matrix_to_align_axes,
    principal_dielectric_tensor
)

%config InlineBackend.figure_format='retina'

Shape of principal dielectric tensors: (4, 4, 4, 3, 3)
Principal dielectric tensor at (0, 0, 0):
 [[2.89 0.   0.  ]
 [0.   2.25 0.  ]
 [0.   0.   2.25]]
Shape of global dielectric tensor: (4, 4, 4, 3, 3)
Global dielectric tensor at (0, 0, 0):
 [[ 0.57510483  2.4122221  -0.09732686]
 [ 2.355218   -0.24285853 -0.0688205 ]
 [-0.53031844 -0.53031844 -2.4788756 ]]


## Example usages of the functions


In [19]:
n_o_array = jnp.ones((4, 4, 4)) * 1.5  # Example 3D array of ordinary refractive indices
n_e_array = jnp.ones((4, 4, 4)) * 1.7  # Example 3D array of extraordinary refractive indices

principal_tensors = principal_dielectric_tensor(n_o_array, n_e_array)
print("Shape of principal dielectric tensors:", principal_tensors.shape)
print("Principal dielectric tensor at (0, 0, 0):\n", principal_tensors[0, 0, 0])


NameError: name 'principal_dielectric_tensor' is not defined

Create components for the optic_axis array

In [ ]:
z, y, x = 4, 4, 4  # Grid size

# For simplicity, use a gradient or a constant array for each component
optic_axis_z = np.ones((z, y, x)) * 0.577  # Example constant value for z-component
optic_axis_y = np.ones((z, y, x)) * 0.577  # Example constant value for y-component
optic_axis_x = np.ones((z, y, x)) * 0.577  # Example constant value for x-component

# Stack the components to create an array of shape (3, z, y, x)
optic_axis = np.stack([optic_axis_z, optic_axis_y, optic_axis_x], axis=0)
epsilon_global = dielectric_tensor_global(n_o_array, n_e_array, optic_axis)
print("Shape of global dielectric tensor:", epsilon_global.shape)
print("Global dielectric tensor at (0, 0, 0):\n", epsilon_global[0, 0, 0])

Steps for forming the global dielectric tensor

In [ ]:
epsilon_prime = principal_dielectric_tensor(n_o_array, n_e_array)

# Rotation matrix from local to global coordinates
rotation_matrices = rotation_matrix_to_align_axes(optic_axis)

### Single optic axis vector

In [3]:
vector = np.array([0.577, 0.577, 0.577])  # Example unit vector
rotation_matrix = rotation_matrix_to_align_axes_single(vector)
print("Rotation matrix:\n", rotation_matrix)
# %%

Rotation matrix:
 [[ 0.57735027  0.40824829 -0.70710678]
 [ 0.57735027  0.40824829  0.70710678]
 [ 0.57735027 -0.81649658  0.        ]]


## Shell sample

In [2]:
# Unpacking data
data = np.load('shell_wide_mla15.npz')
ret = data['ret_image']
azim = data['azim_image']
img_array = data['intensity_images']
birefringence = data['birefingence']
optic_axis = data['optic_axis']

In [4]:
n_o = np.full(birefringence.shape, 1.45)
n_medium = 1.52
wavelength = 0.546
n_e = calc_extraordinary_refractive_index(birefringence, n_o)
print(f"Shape of shell sample: {n_e.shape}")

Shape of shell sample: (9, 29, 29)


In [5]:
epsilon_global = dielectric_tensor_global(n_o, n_e, optic_axis)
non_zero_indices = jnp.nonzero(birefringence)
print(epsilon_global[non_zero_indices])

[[[ 2.003532    0.38648635  0.37467846]
  [ 0.37467846 -2.0666726   0.07006824]
  [ 0.38648635  0.         -2.0666726 ]]

 [[ 2.0736      0.          0.        ]
  [ 0.         -2.1025002   0.        ]
  [ 0.          0.         -2.1025002 ]]

 [[ 2.003532   -0.38648635 -0.37467846]
  [-0.37467846 -2.0666726   0.07006824]
  [-0.38648635  0.         -2.0666726 ]]

 [[ 0.37023556  2.0311809  -0.34088346]
  [ 2.0416958  -0.424531   -0.22992067]
  [-0.2708802  -0.2704947  -2.052731  ]]

 [[ 0.64758897  1.8651971  -0.68695563]
  [ 1.920107   -0.7925381  -0.3088941 ]
  [-0.5138648  -0.51313347 -1.9577755 ]]

 [[ 0.8874201   1.537913   -1.1001333 ]
  [ 1.7222351  -1.1791221  -0.24734102]
  [-0.7806254  -0.77951443 -1.7729156 ]]

 [[ 1.0382767   1.036799   -1.485633  ]
  [ 1.485633   -1.4877504   0.        ]
  [-1.036799   -1.0353234  -1.4877504 ]]

 [[ 1.0892187   0.43591255 -1.7281687 ]
  [ 1.2778308  -1.6228938   0.38854113]
  [-1.2424713  -1.240703   -1.1301191 ]]

 [[ 1.0797185  -0.140347

In [6]:
from chromatix.data.permittivity_tensors import calc_scattering_potential


In [7]:
scattering_potential = calc_scattering_potential(epsilon_global, n_medium, wavelength)
print(f"Shape of scattering potential: {scattering_potential.shape}")

Shape of scattering potential: (9, 29, 29, 3, 3)


Birefringent areas of the sample

In [15]:
print(scattering_potential[non_zero_indices][:5])

[[[  40.637424  -51.18098   -49.617306]
  [ -49.617306  579.6398     -9.278882]
  [ -51.18098     0.        579.6398  ]]

 [[  31.358557    0.          0.      ]
  [   0.        584.3843      0.      ]
  [   0.          0.        584.3843  ]]

 [[  40.637424   51.18098    49.617306]
  [  49.617306  579.6398     -9.278882]
  [  51.18098     0.        579.6398  ]]

 [[ 256.92892  -268.9819     45.141956]
  [-270.37436   362.17697    30.447557]
  [  35.871677   35.82063   577.7936  ]]

 [[ 220.20001  -247.00127    90.97103 ]
  [-254.27278   410.9108     40.90572 ]
  [  68.04925    67.9524    565.21893 ]]]


An area of the sample without any birefringence

In [16]:
print(scattering_potential[0, 0, 0])

[[27.531416  0.        0.      ]
 [ 0.       27.531416  0.      ]
 [ 0.        0.       27.531416]]


This scattering potential represents how the electric field is affected by the sample.